In [15]:
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [16]:
bitstamp_file = "resources/bitstampUSD_1-min_data_2012-01-01_to_2019-08-12.csv"
bitstamp_df = pd.read_csv(bitstamp_file)
bitstamp_df.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
coinbase_file = "resources/coinbaseUSD_1-min_data_2014-12-01_to_2019-01-09.csv"
coinbase_df = pd.read_csv(coinbase_file)
coinbase_df.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1417411980,300.0,300.0,300.0,300.0,0.01,3.0,300.0
1,1417412040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1417412100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1417412160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1417412220,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
crypto_file = "resources/consolidated_coin_data.csv"
crypto_df = pd.read_csv(crypto_file)
crypto_df.head()

,Currency,Date,Open,High,Low,Close,Volume,Market Cap
0,ripple,"Apr 24, 2019",0.321114,0.321282,0.296982,0.302318,"1,517,791,002","12,698,877,293"
1,ripple,"Apr 23, 2019",0.323844,0.328396,0.320919,0.321222,"1,077,333,990","13,492,933,875"
2,ripple,"Apr 22, 2019",0.322277,0.329350,0.320237,0.323934,"1,131,094,080","13,606,823,301"
3,ripple,"Apr 21, 2019",0.328678,0.329627,0.318746,0.322449,"1,005,803,846","13,533,407,430"
4,ripple,"Apr 20, 2019",0.331871,0.333213,0.324969,0.328476,"931,570,799","13,786,384,592"


### Transform bitstamp DataFrame

In [19]:
# Create a copy of the dataframe
bitstamp_transform = bitstamp_df.copy()

# Drop NaN rows
bitstamp_transform = bitstamp_transform.dropna(thresh=2)

# Rename columns to acceptable values within postgres
bitstamp_transform.columns = ['date_bit', 'open_bit', 'high_bit', 'low_bit', 'close_bit', 'volume_btc_bit', 'volume_currency_bit', 'weighted_price_bit']

# Display database
bitstamp_transform

,date_bit,open_bit,high_bit,low_bit,close_bit,volume_btc_bit,volume_currency_bit,weighted_price_bit
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
478,1325346600,4.39,4.39,4.39,4.39,48.000000,210.720000,4.390000
547,1325350740,4.50,4.57,4.50,4.57,37.862297,171.380338,4.526411
548,1325350800,4.58,4.58,4.58,4.58,9.000000,41.220000,4.580000
1224,1325391360,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000
...,...,...,...,...,...,...,...,...
3997692,1565567760,11555.57,11555.57,11540.37,11540.58,0.036868,425.909106,11552.336234
3997693,1565567820,11553.49,11556.22,11553.49,11556.22,0.623462,7204.428272,11555.520505
3997694,1565567880,11559.73,11561.22,11546.77,11561.22,0.159070,1838.731403,11559.252199
3997695,1565567940,11559.73,11589.73,11528.73,11528.73,16.198210,187504.635170,11575.638889


In [20]:
# Resetting index
bitstamp_transform = bitstamp_transform.reset_index()

# Drop columns not being used
bitstamp_transform = bitstamp_transform.drop(columns=['index','volume_btc_bit', 'volume_currency_bit', 'weighted_price_bit'])

# Add column to specify currency
bitstamp_transform['currency_bit'] = 'bitcoin'

# Display database
bitstamp_transform

,date_bit,open_bit,high_bit,low_bit,close_bit,currency_bit
0,1325317920,4.39,4.39,4.39,4.39,bitcoin
1,1325346600,4.39,4.39,4.39,4.39,bitcoin
2,1325350740,4.50,4.57,4.50,4.57,bitcoin
3,1325350800,4.58,4.58,4.58,4.58,bitcoin
4,1325391360,4.58,4.58,4.58,4.58,bitcoin
...,...,...,...,...,...,...
2765814,1565567760,11555.57,11555.57,11540.37,11540.58,bitcoin
2765815,1565567820,11553.49,11556.22,11553.49,11556.22,bitcoin
2765816,1565567880,11559.73,11561.22,11546.77,11561.22,bitcoin
2765817,1565567940,11559.73,11589.73,11528.73,11528.73,bitcoin


In [21]:
# Reorder columns
currency = bitstamp_transform['currency_bit']
bitstamp_transform.drop(labels=['currency_bit'], axis=1, inplace = True)
bitstamp_transform.insert(0, 'currency_bit', currency)

# Transform timestamp into datetime
bitstamp_transform['date_bit'] = pd.to_datetime(bitstamp_transform['date_bit'], unit='s')
bitstamp_transform['date_bit'] = bitstamp_transform['date_bit'].dt.strftime('%b %d, %Y')

# Display database
bitstamp_transform

,currency_bit,date_bit,open_bit,high_bit,low_bit,close_bit
0,bitcoin,"Dec 31, 2011",4.39,4.39,4.39,4.39
1,bitcoin,"Dec 31, 2011",4.39,4.39,4.39,4.39
2,bitcoin,"Dec 31, 2011",4.50,4.57,4.50,4.57
3,bitcoin,"Dec 31, 2011",4.58,4.58,4.58,4.58
4,bitcoin,"Jan 01, 2012",4.58,4.58,4.58,4.58
...,...,...,...,...,...,...
2765814,bitcoin,"Aug 11, 2019",11555.57,11555.57,11540.37,11540.58
2765815,bitcoin,"Aug 11, 2019",11553.49,11556.22,11553.49,11556.22
2765816,bitcoin,"Aug 11, 2019",11559.73,11561.22,11546.77,11561.22
2765817,bitcoin,"Aug 11, 2019",11559.73,11589.73,11528.73,11528.73


In [33]:
# Create a new index to as the id
bitstamp_transform = bitstamp_transform.reset_index()

# Rename index to id
bitstamp_transform = bitstamp_transform.rename(columns = {'index':'id_bit'})

# Set id to index
bitstamp_transform.set_index('id_bit', inplace=True)

# Display database
bitstamp_transform

,currency_bit,date_bit,open_bit,high_bit,low_bit,close_bit
id_bit,,,,,,
0,bitcoin,"Dec 31, 2011",4.39,4.39,4.39,4.39
1,bitcoin,"Dec 31, 2011",4.39,4.39,4.39,4.39
2,bitcoin,"Dec 31, 2011",4.50,4.57,4.50,4.57
3,bitcoin,"Dec 31, 2011",4.58,4.58,4.58,4.58
4,bitcoin,"Jan 01, 2012",4.58,4.58,4.58,4.58
...,...,...,...,...,...,...
2765814,bitcoin,"Aug 11, 2019",11555.57,11555.57,11540.37,11540.58
2765815,bitcoin,"Aug 11, 2019",11553.49,11556.22,11553.49,11556.22
2765816,bitcoin,"Aug 11, 2019",11559.73,11561.22,11546.77,11561.22


### Transform coinbase DataFrame

In [23]:
# Create a copy of the dataframe
coinbase_transform = coinbase_df.copy()

# Drop NaN rows
coinbase_transform = coinbase_transform.dropna(thresh=2)

# Rename columns to acceptable values within postgres
coinbase_transform.columns = ['date_coin', 'open_coin', 'high_coin', 'low_coin', 'close_coin', 'volume_btc_coin', 'volume_currency_coin', 'weighted_price_coin']

# Display dataframe
coinbase_transform

,date_coin,open_coin,high_coin,low_coin,close_coin,volume_btc_coin,volume_currency_coin,weighted_price_coin
0,1417411980,300.00,300.00,300.00,300.00,0.010000,3.000000,300.000000
7,1417412400,300.00,300.00,300.00,300.00,0.010000,3.000000,300.000000
51,1417415040,370.00,370.00,370.00,370.00,0.010000,3.700000,370.000000
77,1417416600,370.00,370.00,370.00,370.00,0.026556,9.825550,370.000000
1436,1417498140,377.00,377.00,377.00,377.00,0.010000,3.770000,377.000000
...,...,...,...,...,...,...,...,...
2099755,1546898520,4006.01,4006.57,4006.00,4006.01,3.382954,13553.433078,4006.390309
2099756,1546898580,4006.01,4006.57,4006.00,4006.01,0.902164,3614.083169,4006.017233
2099757,1546898640,4006.01,4006.01,4006.00,4006.01,1.192123,4775.647308,4006.003635
2099758,1546898700,4006.01,4006.01,4005.50,4005.50,2.699700,10814.241898,4005.719991


In [24]:
# Create a new index to as the id
coinbase_transform = coinbase_transform.reset_index()

# Drop columns not being used
coinbase_transform = coinbase_transform.drop(columns=['index','volume_btc_coin', 'volume_currency_coin', 'weighted_price_coin'])

# Add column to specify currency
coinbase_transform['currency_coin'] = 'bitcoin'

# Display dataframe
coinbase_transform

,date_coin,open_coin,high_coin,low_coin,close_coin,currency_coin
0,1417411980,300.00,300.00,300.00,300.00,bitcoin
1,1417412400,300.00,300.00,300.00,300.00,bitcoin
2,1417415040,370.00,370.00,370.00,370.00,bitcoin
3,1417416600,370.00,370.00,370.00,370.00,bitcoin
4,1417498140,377.00,377.00,377.00,377.00,bitcoin
...,...,...,...,...,...,...
1990686,1546898520,4006.01,4006.57,4006.00,4006.01,bitcoin
1990687,1546898580,4006.01,4006.57,4006.00,4006.01,bitcoin
1990688,1546898640,4006.01,4006.01,4006.00,4006.01,bitcoin
1990689,1546898700,4006.01,4006.01,4005.50,4005.50,bitcoin


In [25]:
# Reorder columns
currency = coinbase_transform['currency_coin']
coinbase_transform.drop(labels=['currency_coin'], axis=1, inplace = True)
coinbase_transform.insert(0, 'currency_coin', currency)

# Transform timestamp into datetime
coinbase_transform['date_coin'] = pd.to_datetime(coinbase_transform['date_coin'], unit='s')
coinbase_transform['date_coin'] = coinbase_transform['date_coin'].dt.strftime('%b %d, %Y')

# Display dataframe
coinbase_transform

,currency_coin,date_coin,open_coin,high_coin,low_coin,close_coin
0,bitcoin,"Dec 01, 2014",300.00,300.00,300.00,300.00
1,bitcoin,"Dec 01, 2014",300.00,300.00,300.00,300.00
2,bitcoin,"Dec 01, 2014",370.00,370.00,370.00,370.00
3,bitcoin,"Dec 01, 2014",370.00,370.00,370.00,370.00
4,bitcoin,"Dec 02, 2014",377.00,377.00,377.00,377.00
...,...,...,...,...,...,...
1990686,bitcoin,"Jan 07, 2019",4006.01,4006.57,4006.00,4006.01
1990687,bitcoin,"Jan 07, 2019",4006.01,4006.57,4006.00,4006.01
1990688,bitcoin,"Jan 07, 2019",4006.01,4006.01,4006.00,4006.01
1990689,bitcoin,"Jan 07, 2019",4006.01,4006.01,4005.50,4005.50


In [26]:
# Create a new index to as the id
coinbase_transform = coinbase_transform.reset_index()

# Rename index to id
coinbase_transform = coinbase_transform.rename(columns = {'index':'id_coin'})

# Set id to index
coinbase_transform.set_index('id_coin', inplace=True)

# Display database
coinbase_transform

,currency_coin,date_coin,open_coin,high_coin,low_coin,close_coin
id_coin,,,,,,
0,bitcoin,"Dec 01, 2014",300.00,300.00,300.00,300.00
1,bitcoin,"Dec 01, 2014",300.00,300.00,300.00,300.00
2,bitcoin,"Dec 01, 2014",370.00,370.00,370.00,370.00
3,bitcoin,"Dec 01, 2014",370.00,370.00,370.00,370.00
4,bitcoin,"Dec 02, 2014",377.00,377.00,377.00,377.00
...,...,...,...,...,...,...
1990686,bitcoin,"Jan 07, 2019",4006.01,4006.57,4006.00,4006.01
1990687,bitcoin,"Jan 07, 2019",4006.01,4006.57,4006.00,4006.01
1990688,bitcoin,"Jan 07, 2019",4006.01,4006.01,4006.00,4006.01


### Transform consolidated Dataframe

In [30]:
# Create a copy of the dataframe
crypto_transform = crypto_df.copy()

# Rename columns to acceptable values within postgres
crypto_transform.columns = ['currency_eth', 'date_eth', 'open_eth', 'high_eth', 'low_eth', 'close_eth', 'volume_eth', 'market_cap']

# Drop all non-ETH cryptocurrencies
eth_df = crypto_transform[crypto_transform['currency_eth'] == 'ethereum']

# Display Dataframe
eth_df

,currency_eth,date_eth,open_eth,high_eth,low_eth,close_eth,volume_eth,market_cap
11648,ethereum,"Apr 24, 2019",171.350000,171.570000,161.980000,165.830000,"7,351,304,156","17,544,029,659"
11649,ethereum,"Apr 23, 2019",172.010000,176.850000,170.900000,171.450000,"7,097,827,092","18,136,462,572"
11650,ethereum,"Apr 22, 2019",170.020000,173.070000,168.730000,171.870000,"6,094,878,077","18,179,292,263"
11651,ethereum,"Apr 21, 2019",173.720000,174.420000,167.430000,170.050000,"6,005,571,688","17,983,602,095"
11652,ethereum,"Apr 20, 2019",173.720000,176.710000,171.790000,173.750000,"6,142,567,812","18,373,023,202"
...,...,...,...,...,...,...,...,...
13000,ethereum,"Aug 11, 2015",0.708087,1.130000,0.663235,1.070000,"1,463,100","64,569,288"
13001,ethereum,"Aug 10, 2015",0.713989,0.729854,0.636546,0.708448,"405,283","42,818,364"
13002,ethereum,"Aug 09, 2015",0.706136,0.879810,0.629191,0.701897,"532,170","42,399,573"
13003,ethereum,"Aug 08, 2015",2.790000,2.800000,0.714725,0.753325,"674,188","45,486,894"


In [31]:
# Reorder dataframe based on date to match other dataframes
eth_df = eth_df.iloc[::-1]

# Create a new index to as the id
eth_df = eth_df.reset_index()

# Drop columns that won't be used
eth_df = eth_df.drop(columns=['index', 'market_cap', 'volume_eth'])

# Display Dataframe
eth_df

,currency_eth,date_eth,open_eth,high_eth,low_eth,close_eth
0,ethereum,"Aug 07, 2015",2.830000,3.540000,2.520000,2.770000
1,ethereum,"Aug 08, 2015",2.790000,2.800000,0.714725,0.753325
2,ethereum,"Aug 09, 2015",0.706136,0.879810,0.629191,0.701897
3,ethereum,"Aug 10, 2015",0.713989,0.729854,0.636546,0.708448
4,ethereum,"Aug 11, 2015",0.708087,1.130000,0.663235,1.070000
...,...,...,...,...,...,...
1352,ethereum,"Apr 20, 2019",173.720000,176.710000,171.790000,173.750000
1353,ethereum,"Apr 21, 2019",173.720000,174.420000,167.430000,170.050000
1354,ethereum,"Apr 22, 2019",170.020000,173.070000,168.730000,171.870000
1355,ethereum,"Apr 23, 2019",172.010000,176.850000,170.900000,171.450000


In [32]:
# Create a new index to as the id
eth_df = eth_df.reset_index()

# Rename index to id
eth_df = eth_df.rename(columns = {'index':'id_eth'})

# Set id to index
eth_df.set_index('id_eth', inplace=True)

# Display database
eth_df

,currency_eth,date_eth,open_eth,high_eth,low_eth,close_eth
id_eth,,,,,,
0,ethereum,"Aug 07, 2015",2.830000,3.540000,2.520000,2.770000
1,ethereum,"Aug 08, 2015",2.790000,2.800000,0.714725,0.753325
2,ethereum,"Aug 09, 2015",0.706136,0.879810,0.629191,0.701897
3,ethereum,"Aug 10, 2015",0.713989,0.729854,0.636546,0.708448
4,ethereum,"Aug 11, 2015",0.708087,1.130000,0.663235,1.070000
...,...,...,...,...,...,...
1352,ethereum,"Apr 20, 2019",173.720000,176.710000,171.790000,173.750000
1353,ethereum,"Apr 21, 2019",173.720000,174.420000,167.430000,170.050000
1354,ethereum,"Apr 22, 2019",170.020000,173.070000,168.730000,171.870000


### Create database connection

In [34]:
connection_string = "postgres:password@localhost:5432/etl_db"
engine = create_engine(f'postgresql://{connection_string}')

In [35]:
engine.table_names()

['bitstamp', 'coinbase', 'eth']

### Load Dataframes into database

In [ ]:
bitstamp_transform.to_sql(name='bitstamp', con=engine, if_exists='append')

In [ ]:
coinbase_transform.to_sql(name='coinbase', con=engine, if_exists='append', index=True)

In [ ]:
eth_df.to_sql(name='eth', con=engine, if_exists='append', index=True)